In [2]:
from llama_index.core import Settings
from llama_index.core.response.notebook_utils import display_response
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.service_context import ServiceContext
from llama_index.core.prompts import PromptTemplate
import os


In [30]:
# llm = Ollama(model="llama3.2-vision:11b", request_timeout=360.0)
# embeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

Settings

In [3]:
storage= 'storage'

In [9]:
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
Settings.llm = Ollama(model="llama3.2:3b", request_timeout=360.0,temperature=0.1)

#


Database

In [6]:
from llama_index.core import  VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

documents = SimpleDirectoryReader('documentacion').load_data()

db= chromadb.PersistentClient(path=storage)

chroma_collection = db.get_or_create_collection('ChatCSV')

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

if not os.path.exists(storage):
    index = VectorStoreIndex.from_documents(documents,show_progress=True, storage_context=storage_context)
else:
    index = VectorStoreIndex.from_vector_store(vector_store,show_progress=True, storage_context=storage_context)


In [7]:

query_engine = index.as_query_engine()



In [13]:
template = (
    "Dado el contexto que te proporcionare responde las preguntas, asi como como debo proceder \n"
    "Contexto:\n"
    "################################\n"
    "{context_str}"
    "################################\n"
    "Responde en español como siendo un ayudante para un operador telefonico del Centro de Servicios Visa: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [14]:
response= query_engine.query('Que debo hacer si el socio realiza un Ip basico mal? Piensa paso a paso tu respuesta')
display_response(response)

**`Final Response:`** Entiendo el contexto. Si el socio realiza un IP básico (IP NO OK) de manera incorrecta, debes seguir estos pasos:

**Paso 1: Verificar la situación**

* Revisa si el socio ya había ingresado desde el IVR como IPB NO OK y si las preguntas de reemplazo fueron realizadas correctamente.
* Si es así, toma como válida la IP.

**Paso 2: Realizar un nuevo módulo de IP (opción a)**

* Si el socio no ha ingresado desde el IVR como IPB NO OK o si las preguntas de reemplazo no fueron realizadas correctamente, debes realizar un nuevo módulo de IP.
* Esto significa que el operador debe solicitar al socio que realice una nueva verificación de la IP.

**Paso 3: Verificar la pregunta del Código de Cuarta Línea**

* Antes de realizar el nuevo módulo de IP, verifica si el socio respondió correctamente a la pregunta del Código de Cuarta Línea.
* Si no lo hizo correctamente, no se realiza un nuevo módulo de IP y se considera como IP NO OK (fin de la gestión).

**Paso 4: Reemplazar preguntas ya realizadas**

* Si el segundo módulo de verificación arroja una pregunta que ya fue realizada en el primer módulo, reemplaza por las preguntas de reemplazo de IPB NO OK (en el IVR o por operador).

**Paso 5: Continuar con la gestión habitual**

* Si el socio supera la IP, continúa con la gestión habitual.

**Paso 6: Procedimiento de IP NO OK para socios UP Scale y Vip**

* Si el socio no supera la IP, debes seguir el procedimiento de IP NO OK para socios UP Scale y Vip (generar el caso Telesoft sin marca S).

Recuerda que es importante verificar cada paso cuidadosamente para asegurarte de que se sigan las instrucciones correctas. ¿Necesitas ayuda adicional?

Contexto utilizado

In [12]:
for _ in response.source_nodes:
    print('X'*50)
    print(_.text)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
URL: https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?&ID=27876

9. IP NO OKAnteIP NO OK se debe aplicar el siguiente procedimiento:-IP NO OK Y TODAS LAS SITUACIONESQUE APLICA MARCA S​​Recordá que para aplicar marca S deberás ingresar en Host desde Menú Principal a opción 6 + 1 + opción 11.Nota: ante IP NO OK no se debe llamar al CAR.SITUACIONES PARTICULARES​RESPUESTA DE TERCEROSNovedadIP NO OK - RESPUESTA DE TERCEROS​.USUARIO VSG O EXT​Gestiones en tarjetas conMARCA S / MARCA A / MARCA L - USUARIO VSG O EXT​.​​TARJETAS SIGNATURE O TARJETAS (CUALQUIER PRODUCTO) CON MARCA I VIP DE 998 O 999 DÍAS​​Para tarjetasSIGNATUREo para tarjetas (cualquier producto) conMARCA I VIP de 998 o 999 días, si el sociono superala IP (IP NO OK) realizá la siguiente gestión:a) Realizá un nuevo módulo de IP(*).(*)Excepción: si el socio respondió de forma incorrecta la pregunta del Código de Cuarta Línea, no se realiza un nuevo módulo de IP y s